In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv("train_18_10000.csv")
df_val = pd.read_csv("val_18_10000.csv")
df_test = pd.read_csv("test_18_10000.csv")

In [ ]:
import numpy as np
import os
import tensorflow as tf

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
train_labels = df_train['label'].values
val_labels = df_val['label'].values
test_labels = df_test['label'].values

df_train = df_train.drop(df_train.columns[0], axis=1)
df_val = df_val.drop(df_val.columns[0], axis=1)
df_test = df_test.drop(df_test.columns[0], axis=1)


df_train = df_train.astype('float32')
df_val = df_val.astype('float32')
df_test = df_test.astype('float32')

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
train = pca.fit_transform(df_train)
val = pca.transform(df_val)
test = pca.transform(df_test)

In [ ]:
df_train.insert(0, 'label', train_labels)
df_val.insert(0, 'label', val_labels)
df_test.insert(0, 'label', test_labels)


In [ ]:
train_labels = np.expand_dims(train_labels, axis=1)
val_labels = np.expand_dims(val_labels, axis=1)
test_labels = np.expand_dims(test_labels, axis=1)

train = np.concatenate((train, train_labels), axis=1)
val = np.concatenate((val, val_labels), axis=1)
test = np.concatenate((test, test_labels), axis=1)

In [ ]:
traindf = pd.DataFrame(train)
valdf = pd.DataFrame(val)
testdf = pd.DataFrame(test)

In [ ]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, InputLayer

label_encoder = LabelEncoder()

dim = train.shape[1] -1

#df_train['label_encoded'] = label_encoder.fit_transform(df_train['512'])
#df_val['label_encoded'] = label_encoder.transform(df_val['512'])
#df_test['label_encoded'] = label_encoder.transform(df_test['512'])

traindf['label_encoded'] = label_encoder.fit_transform(traindf[dim])
valdf['label_encoded'] = label_encoder.transform(valdf[dim])
testdf['label_encoded'] = label_encoder.transform(testdf[dim])

# One-hot encode the integer labels
y_train = to_categorical(traindf['label_encoded'])
y_val = to_categorical(valdf['label_encoded'])
y_test = to_categorical(testdf['label_encoded'])

num_classes = len(traindf['label_encoded'].unique())


In [ ]:
traindf = traindf.drop(columns=[dim])
valdf = valdf.drop(columns=[dim])
testdf = testdf.drop(columns=[dim])

In [ ]:
df_train_1d = np.array(traindf.drop("label_encoded", axis=1)).reshape(60416, 100, 1).astype('float32')
df_val_1d= np.array(valdf.drop("label_encoded", axis=1)).reshape(15104, 100, 1).astype('float32')
df_test_1d = np.array(testdf.drop("label_encoded", axis=1)).reshape(18881, 100, 1).astype('float32')



In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Sequential

model = Sequential()

# Add model layers
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(df_train_1d.shape[1], 1)))
model.add(Conv1D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
import time
start_time = time.time()
model.fit(df_train_1d, y_train, validation_data=(df_val_1d, y_val), epochs=10)
cnn_training_time = time.time() - start_time
print(f"CNN training time: {cnn_training_time:.2f} seconds")

Epoch 1/10
1888/1888 [==============================] - 12s 4ms/step - loss: 0.1177 - accuracy: 0.7215 - val_loss: 0.0770 - val_accuracy: 0.7811
Epoch 2/10
1888/1888 [==============================] - 8s 4ms/step - loss: 0.0752 - accuracy: 0.7853 - val_loss: 0.0740 - val_accuracy: 0.7891
Epoch 3/10
1888/1888 [==============================] - 9s 5ms/step - loss: 0.0708 - accuracy: 0.7978 - val_loss: 0.0713 - val_accuracy: 0.7938
Epoch 4/10
1888/1888 [==============================] - 8s 4ms/step - loss: 0.0672 - accuracy: 0.8081 - val_loss: 0.0704 - val_accuracy: 0.8007
Epoch 5/10
1888/1888 [==============================] - 9s 5ms/step - loss: 0.0643 - accuracy: 0.8163 - val_loss: 0.0696 - val_accuracy: 0.8018
Epoch 6/10
1888/1888 [==============================] - 8s 4ms/step - loss: 0.0623 - accuracy: 0.8223 - val_loss: 0.0698 - val_accuracy: 0.8053
Epoch 7/10
1888/1888 [==============================] - 9s 5ms/step - loss: 0.0604 - accuracy: 0.8286 - val_loss: 0.0686 - val_accuracy

In [ ]:
model.evaluate(df_test_1d, y_test)

591/591 [==============================] - 2s 3ms/step - loss: 0.0687 - accuracy: 0.8051


[0.06868656724691391, 0.8050950765609741]